# LongMem

SERVER HARDWARE SPECIFICATION:
Model: Dell Precision 7920

Processor: Intel Xeon Gold 5218 x 2  (64 Cores)

Memory: 512 GB DDR4 3200 MHz

SSD: Samsung 870 1 TB x 2 RAID 1

GPU: Nvidia RTX 3090 x 2 (nvlink enabled)

In [10]:
#!git clone --recursive https://github.com/pytorch/pytorch.git
#!cd pytorch

In [9]:
#!export CUDA_LAUNCH_BLOCKING=1
#!export TORCH_USE_CUDA_DSA=1

In [9]:
#!python pytorch/setup.py install

In [ ]:
#import torch
#print(torch._C._get_cuda_deterministic_asserts_enabled())
 

In [ ]:
!nvcc --version

In [2]:
!pip install --upgrade torch

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import torch

# CUDA kernel with a device-side assert that will fail
@torch.jit.script
def cuda_kernel_with_assert(x):
    assert x > 0, "Value must be greater than 0."

# Test the CUDA kernel
try:
    x = torch.tensor(-1.0, device="cuda")
    cuda_kernel_with_assert(x)
except RuntimeError as e:
    print("Device-side assert triggered:", e)


Error: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/tmp/ipykernel_439423/4159146384.py", line 6, in cuda_kernel_with_assert
@torch.jit.script
def cuda_kernel_with_assert(x):
    assert x > 0, "Value must be greater than 0."
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
RuntimeError: AssertionError: Value must be greater than 0.


In [1]:
import gc
import torch


gc.collect()


torch.cuda.empty_cache()

In [2]:
!nvidia-smi

Tue Aug  1 13:26:29 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:17:00.0 Off |                  N/A |
|  0%   20C    P8              23W / 350W |  21344MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

The code you provided implements a Long-term Memory Augmented Large Language Model (LLM) that uses a memory bank to cache long-form previous context or knowledge and further takes advantage of it via a decoupled memory module. The LLM model is designed to address the issue of memory staleness in existing large language models.

Here's a summary of what the code is doing:

Import the required libraries and modules.

--Define a CachedMemoryBank class that serves as a memory bank to cache previous context or knowledge. It uses an embedding layer followed by linear layers to create keys and values representations from the input tensor.

--Define a ResidualSideNet class that serves as a decoupled memory module. It takes the transformer outputs (contextual embeddings) as input, applies linear transformations with a ReLU activation function, and then adds a residual connection to the original input. This process allows the model to retrieve and fuse memory-augmented long-context information.

--Define the main LLMWithMemory class, which is the Long-term Memory Augmented Large Language Model. This class combines the CachedMemoryBank and ResidualSideNet modules along with a transformer-based backbone language model (such as GPT-2). The LLMWithMemory class is responsible for memory augmentation and long-form language modeling.

In the forward method of the LLMWithMemory class:
a. The input tensor (input_ids) is passed through the memory bank to generate keys and values representations.
b. The backbone language model is used to obtain transformer outputs (contextual embeddings) for the input tensor.
c. The keys and transformer outputs are multiplied to get memory_attention, which is then softmaxed to obtain attention weights.
d. The attention weights are used to retrieve memory_augmented representations from the memory bank using matrix multiplication.
e. The transformer outputs are reshaped and passed through the ResidualSideNet to get side_net_output.
f. The side_net_output is linearly transformed to predict logits for language modeling.

The generate_text method is provided to generate text from the model. It takes an input tensor and performs a forward pass repeatedly, predicting the next token in a loop until the desired maximum length of the generated text is reached.

The overall goal of this Long-term Memory Augmented Large Language Model is to allow the language model to remember and utilize long-form context or knowledge efficiently, thus improving its language modeling capabilities.

Please note that the code provided may still require further adjustments or fine-tuning based on the specific context and downstream tasks.

In [3]:
import os

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
import torch
import torch.nn as nn
import transformers
from transformers import GPT2Model, GPT2Config
#from torch.nn.parallel import DataParallel
#import torch.distributed as dist
#import torch.multiprocessing as mp


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /root/anaconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/root/anaconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/root/anaconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/root/anaconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/root/anaconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.1

[2023-08-31 12:25:55,747] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-31 12:25:59.306132: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 12:25:59.350923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 12:26:00.185796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
class CachedMemoryBank(nn.Module):
    def __init__(self, vocab_size, embedding_dim, memory_dim):
        super(CachedMemoryBank, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.memory_key = nn.Linear(embedding_dim, memory_dim)
        self.memory_value = nn.Linear(embedding_dim, memory_dim)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        keys = self.memory_key(embedded)
        values = self.memory_value(embedded)
        return keys, values


class ResidualSideNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(ResidualSideNet, self).__init__()

        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, input_dim)

    def forward(self, input_tensor):
        residual = input_tensor  # Store the input tensor for the residual connection
        output = self.fc1(input_tensor)
        output = self.relu(output)
        output = self.fc2(output)
        output += residual  # Add residual connection
        return output
    
class LLMmodel():
        def __init__(self, modelname):
            super(LLMmodel, self).__init__()
            self.modelname= modelname
            tokenizer = LlamaTokenizer.from_pretrained(modelname,trust_remote_code=True)
            self.tokenizer = tokenizer

        
        def simpleLLM(self):
            #bnb_config = BitsAndBytesConfig(
            
            #bnb_4bit_quant_type="nf4",
            
            #bnb_4bit_compute_dtype=torch.float16,
            #)
           #model= LlamaForCausalLM.from_pretrained(self.modelname,
           #                                 load_in_8bit=True,
           #                                 device_map='auto', quantization_config= bnb_config,
           #                                 torch_dtype=torch.float16,
           #                                 offload_folder = 'model')
            model= LlamaForCausalLM.from_pretrained(self.modelname,
                                            load_in_8bit=True,
                                            device_map='auto', 
                                            torch_dtype=torch.float16,
                                            offload_folder = 'model')
            model.config.use_cache = False
            # Move the model and memory bank to the GPU
            #device = torch.device("cuda:0")
            #model = model.to(device)
            # Use DataParallel to replicate the model across multiple GPUs
            #model = DataParallel(model)
            # Set the environment variables for distributed training
            #os.environ["MASTER_ADDR"] = "127.0.0.1"  # Replace with the actual IP address or hostname of the master process
            #os.environ["MASTER_PORT"] = "12345"      # Replace with the actual port number used by the master process

            world_size = 2  # Number of processes/GPUs to use
            rank=2
            # Set the distributed environment variables
            #dist.init_process_group(backend='nccl', rank=rank, world_size=world_size)

            # Use DistributedDataParallel for zero sharding
            #model = nn.parallel.DistributedDataParallel(model, device_ids=[rank])

            self.model= model
            return model
        def LLM(self):
            #Set Dataset , Model Name and Parameter accordin to your Large Language model
            dataset = self.dataset
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            try:
                tokenizer=self.tokenizer
                tokenizer.pad_token = tokenizer.eos_token
                model = LlamaForCausalLM.from_pretrained(self.modelname,
                                                 quantization_config=bnb_config,
                                                trust_remote_code=True,
                                                 load_in_4bit=True,
                                                 device_map='auto',
                                                 torch_dtype=torch.float16,
                                                 offload_folder='model')
                model.config.use_cache = False
                #model.train()
                lora_alpha = 16
                lora_dropout = 0.1
                lora_r = 64
                peft_config = LoraConfig(
                    lora_alpha=lora_alpha,
                    lora_dropout=lora_dropout,
                    r=lora_r,
                    bias="none",
                    task_type="CAUSAL_LM",)
                output_dir = "./results"
                micro_batch_size = 4
                gradient_accumulation_steps = 4
                optim = "paged_adamw_32bit"
                save_steps = 10
                logging_steps = 10
                learning_rate = 2e-4
                max_grad_norm = 0.3
                max_steps = 5
                warmup_ratio = 0.03
                val_set_size=0.2
                lr_scheduler_type = "constant"
                training_arguments=transformers.TrainingArguments(
                    per_device_train_batch_size=micro_batch_size,
                    gradient_accumulation_steps=gradient_accumulation_steps,
                    warmup_steps=100,
                    num_train_epochs=num_epochs,
                    learning_rate=learning_rate,
                    fp16=True,
                    logging_steps=10,
                    optim="paged_adamw_8bit",
                    evaluation_strategy="steps",
                    save_strategy="steps",
                    eval_steps=200,
                    save_steps=200,
                    output_dir=output_dir,
                    save_total_limit=3
                )
            #trainer = transformers.Trainer(
            #model=model,
            #    train_dataset=train_data,
            #eval_dataset=val_data,
            #args=training_arguments,
            #    data_collator=transformers.DataCollatorForSeq2Seq(
            #       tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
            #  ),
             #  ) ''''
                from trl import SFTTrainer
                max_seq_length = 512
                train = dataset['train']      # len(train)=84437 (95%)
                val = dataset['validation']   # len(val)=4401 (5%)
                trainer = SFTTrainer(
                    model=model,
                    train_dataset=train,
                    peft_config=peft_config,
                    dataset_text_field="text",
                    max_seq_length=max_seq_length,
                    tokenizer=tokenizer,
                    args=training_arguments,
                )
            #trainer.train()
            #model_save = trainer.model.module if hasattr(trainer.model, "module") else trainer.model
            #model_save.save_pretrained("japanese-mc4-falcon")
                self.model = model
                return model
            except Exception as e:
                print("Error loading the model:", e)
                return None, None
        
class LLMWithMemory(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, memory_dim,modelname):
        super(LLMWithMemory, self).__init__()
       # Move the model and memory bank to the GPU
        #device = torch.device("cuda:0")
        self.memory_bank = CachedMemoryBank(vocab_size, embedding_dim, memory_dim)
        model= GPT2Model.from_pretrained(modelname) 
        #device = torch.device("cuda:0")
        #model = model.to(device)
        #modelclass.modelname= "TheBloke/stable-vicuna-13B-HF"
        #model = modelclass.simpleLLM()
        self.transformer = model
        self.side_net = ResidualSideNet(hidden_dim, memory_dim)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        # Move the model to the GPU if available
        if torch.cuda.is_available():
            self.cuda()

    def forward(self, input_ids, hidden=None):
        keys, values = self.memory_bank(input_ids)
        transformer_outputs = self.transformer(input_ids)[0]
        
        # Convert keys and values to float16 (matching the model's requirement)
        keys = keys.float()
        values = values.float()

        # Memory augmentation
        memory_attention = torch.matmul(keys, transformer_outputs.transpose(-1, -2))
        memory_attention = memory_attention.transpose(-1, -2)

        # Apply the memory-augmented input to the SideNet
        memory_augmented = torch.matmul(memory_attention, values)
        transformer_outputs = transformer_outputs + memory_augmented
        # Apply side_net to transformer_outputs
        side_net_output = self.side_net(transformer_outputs.view(-1, transformer_outputs.size(-1)))
        side_net_output = side_net_output.view(transformer_outputs.size(0), transformer_outputs.size(1), -1)
       #print("transformer output",transformer_outputs.shape,transformer_outputs.type)
        #print("keys",keys.shape,keys.type)
        #print("values", values.shape, values.type)
        logits = self.linear(side_net_output)
        return logits, keys, values, memory_augmented
    
    def generate_text(self, input_ids, max_length, hidden=None):
        with torch.no_grad():
            # Create a copy of the input_ids tensor to store the generated tokens
            batch_size = input_ids.size(0)
            #input_ids = input_ids.to('cuda')  # Assuming you have input_ids tensor
            output_ids = input_ids.clone()
            
            for _ in range(max_length):
                # Call the forward method with the current output_ids
                # Inside your training loop:
                scaler = GradScaler()
                with autocast():
                    logits, _, _, _ = self.forward(output_ids)
                    loss = compute_loss(logits, target)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                # Get the predicted token for the next step
                predicted_token = torch.argmax(logits[:, -1, :], dim=-1)
                # Append the predicted token to the output_ids
                output_ids = torch.cat((output_ids, predicted_token.unsqueeze(1)), dim=1)
        return output_ids,predicted_token 

print(torch.cuda.is_available())
# Example usage
vocab_size = 100000
embedding_dim = 768
hidden_dim = 768
num_layers = 12
memory_dim = 768
modelname= "gpt2"
model = LLMWithMemory(vocab_size, embedding_dim, hidden_dim, num_layers, memory_dim,modelname)
#model = model.to("cuda:0")  


True


KeyboardInterrupt: 

In [6]:
model.eval()

LLMWithMemory(
  (memory_bank): CachedMemoryBank(
    (embedding): Embedding(100000, 768)
    (memory_key): Linear(in_features=768, out_features=768, bias=True)
    (memory_value): Linear(in_features=768, out_features=768, bias=True)
  )
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), ep

In [17]:
# Save the model's state dictionary to a file
#torch.save(model.state_dict(), "llm_model_state_dict.pth")

In [10]:
import os
import torch
from torch.cuda.amp import autocast, GradScaler

def generate_text_with_gpu_handling(model, input_ids, max_length, batch_size=32, cache_folder="./cached_data"):
    with torch.no_grad():
        #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        #model = model.to(device)
        
        os.makedirs(cache_folder, exist_ok=True)
        
        # Function to load cached data from disk
        def load_cached_data(batch_index):
            filename = os.path.join(cache_folder, f"batch_{batch_index}.pt")
            return torch.load(filename)

        # Function to save data to disk
        def save_cached_data(batch_index, data):
            filename = os.path.join(cache_folder, f"batch_{batch_index}.pt")
            torch.save(data, filename)

        output_ids = input_ids.clone()
        total_batches = len(input_ids) // batch_size + 1

        for batch_index in range(total_batches):
            start_index = batch_index * batch_size
            end_index = (batch_index + 1) * batch_size
            batch_input_ids = input_ids[start_index:end_index].to(device)

            # Check if data is already cached on disk
            if os.path.exists(os.path.join(cache_folder, f"batch_{batch_index}.pt")):
                cached_data = load_cached_data(batch_index)
                batch_input_ids = cached_data["batch_input_ids"]
                batch_output_ids = cached_data["batch_output_ids"]
            else:
                batch_output_ids = batch_input_ids.clone()
                for _ in range(max_length):
                    with autocast():
                        logits, _, _, _ = model(batch_input_ids)
                        predicted_token = torch.argmax(logits[:, -1, :], dim=-1)
                        batch_input_ids = torch.cat((batch_input_ids, predicted_token.unsqueeze(1)), dim=1)

                # Cache the batch data to disk
                save_cached_data(batch_index, {"batch_input_ids": batch_input_ids, "batch_output_ids": batch_output_ids})

            output_ids[start_index:end_index] = batch_output_ids

        return output_ids

In [ ]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [11]:
import PyPDF2

def pdf_to_text(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        num_pages = pdf_reader.numPages

        for page_num in range(num_pages):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()

    return text

#if __name__ == "__main__":
#    pdf_file_path = "demodoc.pdf"
#    extracted_text = pdf_to_text(pdf_file_path)
#    print(extracted_text)


In [12]:
from transformers import GPT2Tokenizer

# Instantiate the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

pdf_file_path = "demodoc.pdf"
extracted_text = pdf_to_text(pdf_file_path)
# Example text input
text_input = extracted_text[1:4000]
# Tokenize the text input
input_ids = tokenizer.encode(text_input, return_tensors="pt")
# Set the maximum length you want to generate
max_length = 100
output_ids = generate_text_with_gpu_handling(model, input_ids, max_length, batch_size=32)
print("generated_output ",generated_output)

Token indices sequence length is longer than the specified maximum sequence length for this model (1289 > 1024). Running this sequence through the model will result in indexing errors


NameError: name 'device' is not defined

In [11]:
#modelclass = LLMmodel(modelname)
# Instantiate the tokenizer
tokenizer = LlamaTokenizer.from_pretrained(modelname,trust_remote_code=True)
pdf_file_path = "demodoc.pdf"
extracted_text = pdf_to_text(pdf_file_path)
# Example text input
text_input = extracted_text[1:5000]
# Tokenize the text input
input_ids = tokenizer.encode(text_input, return_tensors="pt")

output_logits, keys, values, memory_augmented = model(input_ids)

print("Output logits shape:", output_logits.shape)
print("Keys shape:", keys.shape)
print("Values shape:", values.shape)
print("Memory augmented shape:", memory_augmented.shape)
# Generate text
max_length = 20
#generated_text = model.generate_text(input_ids, max_length)
# Get the predicted token for the next step
predicted_token = torch.argmax(output_logits[:, -1, :], dim=-1)
# Append the predicted token to the output_ids
output_ids = torch.cat((input_ids, predicted_token.unsqueeze(1)), dim=1)

text= output_ids
print("Generated Ids:", output_ids)
print("predicted_token:", predicted_token)
# Verify the token IDs (output_ids)
print("Output IDs:", output_ids)

# Check if the tokenizer is loaded correctly
print("Tokenizer:", tokenizer)

# Verify the tokenizer vocabulary size
print("Vocabulary Size:", tokenizer.vocab_size)
# Decode generated token IDs to text
# ... (Previous code remains the same)

# Decode generated token IDs to text, skipping special tokens and handling out-of-range token IDs
generated_text_tokens = []
for token_id in output_ids.squeeze():
    if token_id < tokenizer.vocab_size:
        print(tokenizer.decode(token_id.item(), skip_special_tokens=True))
        generated_text_tokens.append(tokenizer.decode(token_id.item(), skip_special_tokens=True))
    else:
        # Handle out-of-range token ID
        generated_text_tokens.append("<unk>")

# Concatenate the tokens to form the generated text
generated_text = " ".join(generated_text_tokens)

print("Generated text:", generated_text)

#generated_text = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=True)

#print("Generated text:", generated_text)

OutOfMemoryError: CUDA out of memory. Tried to allocate 52.00 MiB (GPU 0; 23.69 GiB total capacity; 15.22 GiB already allocated; 47.38 MiB free; 15.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # Assuming you have one GPU (change 0 to the appropriate GPU number if needed)
import os

os.environ["TORCH_USE_CUDA_DSA"] = "0"  # Disable device-side assertions

modelclass = LLMmodel(modelname)
# Instantiate the tokenizer
tokenizer = modelclass.tokenizer

# Example text input
text_input = "Once upon a time"

# Tokenize the text input
input_ids = tokenizer.encode(text_input, return_tensors="pt")
input_ids= input_ids[:, :5]
input_ids

print("Output logits shape:", output_logits.shape)
print("Keys shape:", keys.shape)
print("Values shape:", values.shape)
print("Memory augmented shape:", memory_augmented.shape)
# Generate text
max_length = 20
generated_ids = model.generate_text(input_ids, max_length)
print("Generated ids:", generated_id)
# Decode generated token IDs to text
generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)

print("Generated text:", generated_text)